In [138]:
import os, sys, json, time
from pathlib import Path
import numpy as np
import pandas as pd

ROOT = os.path.abspath("..")
if ROOT not in sys.path:
    sys.path.insert(0, ROOT)

with open(os.path.join(ROOT, "config.json"), "r") as f:
    C = json.load(f)

TICKER   = C["ticker"]; START = C["start"]; END = C["end"]; INTERVAL = C["interval"]
HORIZON  = int(C["horizon"]); LOOKBACK = int(C["lookback"])
BATCH    = int(C["batch"]);   EPOCHS   = int(C["epochs"])
SEED     = int(C.get("seed", 42))

RESULTS_DIR = Path(C.get("results_dir", "../results"))
RESULTS_DIR.mkdir(parents=True, exist_ok=True)
RUN_DIR   = RESULTS_DIR / time.strftime("%Y-%m-%d_%H-%M-%S_lstm")
RUN_DIR.mkdir(parents=True, exist_ok=True)   # <- hinzufügen
print("RUN_DIR:", RUN_DIR)                   # optional

TRAIN_CSV = f"../data/{TICKER}_{INTERVAL}_{START}_{END}_cls_h{HORIZON}.csv"

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, balanced_accuracy_score, matthews_corrcoef, average_precision_score
import joblib

np.random.seed(SEED)
tf.random.set_seed(SEED)


RUN_DIR: ..\results\2025-10-03_00-36-36_lstm


In [139]:
df = pd.read_csv(TRAIN_CSV, index_col=0, parse_dates=[0]).sort_index()

exp = {"open","high","low","close","volume","logret_1d","target"}
missing = exp - set(df.columns)
assert not missing, f"Fehlende Spalten: {missing}"
assert not df.index.has_duplicates
assert (df["close"] > 0).all()
assert df.notna().all().all()


In [140]:
# === 3) Features / Ziel wählen (robust) ===
import os, yaml

OHLCV = {"open","high","low","close","volume"}

# Kandidaten aus DF: alles außer OHLCV + target
candidates_from_df = [c for c in df.columns if c not in OHLCV | {"target"}]

# Optional: YAML lesen und mit DF schneiden
features_yaml = "../data/features_v1.yml"
if os.path.exists(features_yaml):
    with open(features_yaml, "r") as f:
        meta = yaml.safe_load(f)
    yaml_feats = meta.get("features", [])
    FEATURES = [c for c in yaml_feats if c in candidates_from_df]
    # Fallback, falls YAML-Features (noch) nicht alle im CSV sind:
    if not FEATURES:
        FEATURES = candidates_from_df
else:
    FEATURES = candidates_from_df

# Sicherheit: nur numerische Spalten nehmen
FEATURES = [c for c in FEATURES if pd.api.types.is_numeric_dtype(df[c])]

assert len(FEATURES) > 0, "Keine nutzbaren Features gefunden – prüfe Block 2 Export."
print("FEATURES:", FEATURES)

TARGET = "target"
X = df[FEATURES].copy()
y = df[TARGET].astype(int).copy()


FEATURES: ['logret_1d', 'vola_10d', 'sma_10', 'sma_20', 'sma_diff']


In [141]:
# === 4) Chronologische Splits (70/15/15) ===
n = len(df)
n_train = int(n * 0.70)
n_val   = int(n * 0.15)
n_test  = n - n_train - n_val

train_idx = slice(0, n_train)
val_idx   = slice(n_train, n_train + n_val)
test_idx  = slice(n_train + n_val, n)

X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
X_val,   y_val   = X.iloc[val_idx],   y.iloc[val_idx]
X_test,  y_test  = X.iloc[test_idx],  y.iloc[test_idx]

print(f"Split sizes → train {len(X_train)}, val {len(X_val)}, test {len(X_test)}")

Split sizes → train 2391, val 512, test 513


In [142]:
print("X_train shape/check:", X_train.shape, " | cols:", list(X_train.columns))


X_train shape/check: (2391, 5)  | cols: ['logret_1d', 'vola_10d', 'sma_10', 'sma_20', 'sma_diff']


In [143]:
# === 5) Scaler nur auf TRAIN fitten ===
scaler = StandardScaler(with_mean=True, with_std=True)
X_train_s = pd.DataFrame(scaler.fit_transform(X_train), index=X_train.index, columns=FEATURES)
X_val_s   = pd.DataFrame(scaler.transform(X_val),       index=X_val.index,   columns=FEATURES)
X_test_s  = pd.DataFrame(scaler.transform(X_test),      index=X_test.index,  columns=FEATURES)

# Scaler speichern (für spätere Runs/Inference)
import joblib, io
joblib.dump(scaler, RUN_DIR / "scaler.joblib")

['..\\results\\2025-10-03_00-36-36_lstm\\scaler.joblib']

In [144]:
def drift_summary(Xa: pd.DataFrame, Xb: pd.DataFrame):
    out = []
    for c in Xa.columns:
        m1, s1 = Xa[c].mean(), Xa[c].std(ddof=1)
        m2, s2 = Xb[c].mean(), Xb[c].std(ddof=1)
        ratio_std = float((s2 + 1e-9) / (s1 + 1e-9))
        diff_mean = float(m2 - m1)
        out.append({"feature": c, "mean_diff": diff_mean, "std_ratio": ratio_std})
    return pd.DataFrame(out).sort_values("std_ratio", ascending=False)

drift_df = drift_summary(X_train_s, X_test_s)
drift_df.to_csv(RUN_DIR / "drift_train_vs_test.csv", index=False)
print(drift_df.head())


     feature  mean_diff  std_ratio
1   vola_10d  -0.104683   0.999691
0  logret_1d  -0.025615   0.956340
4   sma_diff  -0.139882   0.915933
2     sma_10   5.075310   0.730956
3     sma_20   5.125317   0.724050


In [145]:
# === 6) Windowing: Sequenzen der Länge LOOKBACK → Label am Endzeitpunkt ===
def make_windows(X_df: pd.DataFrame, y_ser: pd.Series, lookback: int):
    X_values = X_df.values.astype(np.float32)
    y_values = y_ser.values.astype(np.int32)
    n = len(X_df)
    xs, ys = [], []
    for i in range(lookback-1, n):
        xs.append(X_values[i - lookback + 1 : i + 1])  # inkl. i
        ys.append(y_values[i])                          # Label für Zeitpunkt i (Up/Down für i->i+H)
    return np.stack(xs, axis=0), np.array(ys)

Xtr_win, ytr = make_windows(X_train_s, y_train, LOOKBACK)
Xva_win, yva = make_windows(X_val_s,   y_val,   LOOKBACK)
Xte_win, yte = make_windows(X_test_s,  y_test,  LOOKBACK)

print("Shapes:",
      "\n  train:", Xtr_win.shape, ytr.shape,
      "\n  val  :", Xva_win.shape, yva.shape,
      "\n  test :", Xte_win.shape, yte.shape)

Shapes: 
  train: (2332, 60, 5) (2332,) 
  val  : (453, 60, 5) (453,) 
  test : (454, 60, 5) (454,)


In [146]:
# === class_weight (optional) aus Trainingslabels berechnen ===
from collections import Counter
cw = None
counts = Counter(ytr.tolist())
if len(counts) == 2:
    total = sum(counts.values())
    # einfache Invers-Häufigkeit (normalisiert), robust bei leichter Schieflage
    cw = {0: total/(2*counts.get(0, 1)), 1: total/(2*counts.get(1, 1))}
print("class_weight:", cw)


class_weight: {0: 1.052346570397112, 1: 0.9526143790849673}


In [147]:
# === 7) tf.data Pipelines ===
def to_ds(X, y, batch, shuffle):
    ds = tf.data.Dataset.from_tensor_slices((X, y))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(X), seed=SEED, reshuffle_each_iteration=True)
    return ds.batch(batch).prefetch(tf.data.AUTOTUNE)

ds_train = to_ds(Xtr_win, ytr, BATCH, shuffle=True)
ds_val   = to_ds(Xva_win, yva, BATCH, shuffle=False)
ds_test  = to_ds(Xte_win, yte, BATCH, shuffle=False)

In [148]:
# Basis-Rate im Training (für Output-Bias)
pos_rate_train = float(ytr.mean())
from math import log
def _logit(p): 
    eps = 1e-6
    p = min(max(p, eps), 1-eps)
    return log(p/(1-p))
output_bias_init = tf.keras.initializers.Constant(_logit(pos_rate_train))
print("pos_rate_train:", round(pos_rate_train,3))


pos_rate_train: 0.525


In [149]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, matthews_corrcoef

logit = LogisticRegression(max_iter=200, n_jobs=None)
logit.fit(X_train_s.iloc[LOOKBACK-1:], y_train.iloc[LOOKBACK-1:])  # grob: letztes Fensterende
y_proba_lr = logit.predict_proba(X_test_s.iloc[LOOKBACK-1:])[:,1]
print("LogReg AUROC:", round(roc_auc_score(y_test.iloc[LOOKBACK-1:], y_proba_lr), 3))
print("LogReg MCC@0.5:", round(matthews_corrcoef(y_test.iloc[LOOKBACK-1:], (y_proba_lr>=0.5).astype(int)), 3))


LogReg AUROC: 0.472
LogReg MCC@0.5: -0.052


In [150]:
# === 8) Modell definieren ===
n_features = Xtr_win.shape[-1]

model = keras.Sequential([
    layers.Input(shape=(LOOKBACK, n_features)),
    layers.LSTM(64, return_sequences=True),
    layers.LayerNormalization(),          # NEU: stabilisiert Sequenzstatistiken
    layers.Dropout(0.1),                  # weniger Dropout (0.2 -> 0.1)
    layers.LSTM(32),
    layers.LayerNormalization(),          # NEU
    layers.Dense(16, activation="relu"),
    layers.Dense(1, activation="sigmoid", 
                 bias_initializer=output_bias_init),  # NEU: sinnvoller Startpunkt
])


model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=5e-4),  # 1e-3 -> 5e-4
    loss=keras.losses.BinaryCrossentropy(),
    metrics=[
        keras.metrics.BinaryAccuracy(name="acc"),
        keras.metrics.AUC(name="auc"),
        keras.metrics.AUC(name="auprc", curve="PR"),
        keras.metrics.Precision(name="prec"),
        keras.metrics.Recall(name="rec"),
    ],
)

ckpt_path = RUN_DIR / "best.keras"
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath=str(ckpt_path),
        monitor="val_auprc", mode="max", save_best_only=True, verbose=1
    ),
    keras.callbacks.EarlyStopping(
        monitor="val_auprc", mode="max", patience=12, restore_best_weights=True
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_auprc", mode="max", factor=0.5, patience=6, min_lr=1e-5, verbose=1
    ),
]


model.summary()

Model: "sequential_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_18 (LSTM)                       │ (None, 60, 64)              │          17,920 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ layer_normalization_2                │ (None, 60, 64)              │             128 │
│ (LayerNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 60, 64)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_19 (LSTM)                       │ (None, 32)                  │          12,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ layer_normalization_3                │ (None, 32)                  │              64 │
│ (LayerNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_18 (Dense)                     │ (None, 16)                  │             528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_19 (Dense)                     │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 31,073 (121.38 KB)

 Trainable params: 31,073 (121.38 KB)

 Non-trainable params: 0 (0.00 B)

In [151]:
# === 9) Callbacks ===
ckpt_path = RUN_DIR / "best.keras"
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath=str(ckpt_path),
        monitor="val_auc",
        mode="max",
        save_best_only=True,
        save_weights_only=False,
        verbose=1,
    ),
    keras.callbacks.EarlyStopping(
        monitor="val_auc", mode="max", patience=10, restore_best_weights=True
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_auc", mode="max", factor=0.5, patience=5, verbose=1
    ),
]

In [152]:
# === 10) Train ===
history = model.fit(
    ds_train,
    validation_data=ds_val,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=1,
    class_weight=cw  # NEU
)


# Trainingskurve speichern
pd.DataFrame(history.history).to_csv(RUN_DIR / "history.csv", index=False)

Epoch 1/100
33/37 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - acc: 0.5141 - auc: 0.5159 - auprc: 0.5372 - loss: 0.7337 - prec: 0.5302 - rec: 0.5659
Epoch 1: val_auc improved from None to 0.49791, saving model to ..\results\2025-10-03_00-36-36_lstm\best.keras
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step - acc: 0.5017 - auc: 0.4991 - auprc: 0.5311 - loss: 0.7167 - prec: 0.5258 - rec: 0.5163 - val_acc: 0.5121 - val_auc: 0.4979 - val_auprc: 0.5138 - val_loss: 0.6978 - val_prec: 0.5103 - val_rec: 0.9739 - learning_rate: 5.0000e-04
Epoch 2/100
35/37 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - acc: 0.5269 - auc: 0.5314 - auprc: 0.5625 - loss: 0.6967 - prec: 0.5490 - rec: 0.6127
Epoch 2: val_auc improved from 0.49791 to 0.50060, saving model to ..\results\2025-10-03_00-36-36_lstm\best.keras
37/37 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - acc: 0.5300 - auc: 0.5350 - auprc: 0.5563 - loss: 0.6973 - prec: 0.5496 - rec: 0.5792 - val_acc: 0.5011 - val_auc: 0.5006 - val_auprc: 0.5265 - val_loss: 0.7046 - val_prec: 0.5046 - val

In [153]:
# === env_info & model_card (Reproduzierbarkeit/Doku) ===
env_info = {
    "python": sys.version,
    "tensorflow": tf.__version__,
    "numpy": np.__version__,
    "pandas": pd.__version__,
    "seed": SEED,
    "lookback": LOOKBACK,
    "features": ["logret_1d"],  # falls du später mehr nutzt, hier dynamisieren
    "batch": BATCH,
    "epochs": EPOCHS,
    "optimizer": "Adam(1e-3)",
    "loss": "BinaryCrossentropy",
    "metrics": ["acc","auc","auprc","prec","rec"]
}
with open(RUN_DIR / "env_info.json", "w") as f:
    json.dump(env_info, f, indent=2)

with open(RUN_DIR / "model_card.md", "w", encoding="utf-8") as f:
    f.write(
        "# LSTM-Klassifikator\n\n"
        f"- Input: Window={LOOKBACK} x Features={env_info['features']}\n"
        "- Loss: Binary Cross-Entropy\n"
        "- Optimizer: Adam(1e-3)\n"
        "- Metrics: acc, auc, auprc, precision, recall\n"
        f"- class_weight: {cw}\n"
        "- Notes: label = 1 wenn close(t+H) > close(t) (log-return-basiert)\n"
    )


In [154]:
# === 11) Evaluate & Berichte ===
# Best Weights sind dank EarlyStopping bereits geladen
test_metrics = model.evaluate(ds_test, return_dict=True, verbose=0)
print("Test metrics:", json.dumps(test_metrics, indent=2))

# Schwellenwert 0.5 (später kalibrierbar)
y_proba = model.predict(ds_test, verbose=0).ravel()
y_pred  = (y_proba >= 0.5).astype(int)

print("\nConfusion matrix (test):\n", confusion_matrix(yte, y_pred))
print("\nClassification report (test):\n", classification_report(yte, y_pred, digits=3))

Test metrics: {
  "acc": 0.46035242080688477,
  "auc": 0.5008494853973389,
  "auprc": 0.5273241996765137,
  "loss": 0.7126410603523254,
  "prec": 0.5,
  "rec": 0.01224489789456129
}

Confusion matrix (test):
 [[206   3]
 [242   3]]

Classification report (test):
               precision    recall  f1-score   support

           0      0.460     0.986     0.627       209
           1      0.500     0.012     0.024       245

    accuracy                          0.460       454
   macro avg      0.480     0.499     0.325       454
weighted avg      0.482     0.460     0.302       454



In [155]:
# Val-basierte Schwelle (max MCC, Korridor) direkt in Block 3
from sklearn.metrics import matthews_corrcoef, roc_auc_score
import numpy as np

val_proba = model.predict(ds_val, verbose=0).ravel()

def choose_threshold(y_true, y_prob, bounds=(0.2, 0.8)):
    uniq = np.unique(y_prob); cand = np.r_[0.0, uniq, 1.0]
    best_t, best_s = 0.5, -1
    for t in cand:
        yp = (y_prob >= t).astype(int)
        pr = yp.mean()
        if not (bounds[0] <= pr <= bounds[1]): 
            continue
        s = matthews_corrcoef(y_true, yp)
        if s > best_s: best_s, best_t = s, float(t)
    return best_t

thr = choose_threshold(yva, val_proba, bounds=(0.2,0.8))
y_pred_thr = (y_proba >= thr).astype(int)

print(f"\n[Block3 quick] thr(val@maxMCC)={thr:.3f} | AUROC val/test={roc_auc_score(yva, val_proba):.3f}/{roc_auc_score(yte, y_proba):.3f}")
print("CM(test@thr):\n", confusion_matrix(yte, y_pred_thr))
print(classification_report(yte, y_pred_thr, digits=3))



[Block3 quick] thr(val@maxMCC)=0.485 | AUROC val/test=0.547/0.498
CM(test@thr):
 [[202   7]
 [241   4]]
              precision    recall  f1-score   support

           0      0.456     0.967     0.620       209
           1      0.364     0.016     0.031       245

    accuracy                          0.454       454
   macro avg      0.410     0.491     0.325       454
weighted avg      0.406     0.454     0.302       454



In [156]:
# --- Diagnose der Probabilitäten ---
import numpy as np
from sklearn.metrics import roc_auc_score

print("Proba stats  (test): min=", float(y_proba.min()), 
      "max=", float(y_proba.max()), "mean=", float(y_proba.mean()))

# AUC auf VAL & TEST (Ranking-Qualität, unabhängig vom Threshold)
val_proba = model.predict(ds_val, verbose=0).ravel()
print("AUROC val/test:", 
      round(roc_auc_score(yva, val_proba), 3), "/", 
      round(roc_auc_score(yte, y_proba), 3))

# Quick check: Ist das Signal invertiert?
if roc_auc_score(yva, val_proba) < 0.5:
    print("⚠️ AUROC < 0.5 auf VAL → Versuch: invertiere Scores (1-p)")
    y_proba_inverted = 1.0 - y_proba
    from sklearn.metrics import classification_report, confusion_matrix
    y_pred_inv = (y_proba_inverted >= 0.5).astype(int)
    print("Confusion (inv, thr=0.5):\n", confusion_matrix(yte, y_pred_inv))
    print("Report (inv):\n", classification_report(yte, y_pred_inv, digits=3))


Proba stats  (test): min= 0.3792177140712738 max= 0.5352762937545776 mean= 0.43799129128456116
AUROC val/test: 0.547 / 0.498


In [157]:
# === Extra Test-Metriken ===
bal_acc = balanced_accuracy_score(yte, y_pred)
mcc = matthews_corrcoef(yte, y_pred)
auprc_test = average_precision_score(yte, y_proba)  # probabilistische PR-Qualität

extra = {
    "balanced_accuracy": float(bal_acc),
    "mcc": float(mcc),
    "auprc": float(auprc_test)
}
print("Extra test metrics:", json.dumps(extra, indent=2))

# persistieren
with open(RUN_DIR / "extra_test_metrics.json", "w") as f:
    json.dump(extra, f, indent=2)


Extra test metrics: {
  "balanced_accuracy": 0.4989454154867689,
  "mcc": -0.009205617762249547,
  "auprc": 0.5283307301859568
}


In [158]:
# === 12) Artefakte sichern ===
# Keras-Format (SavedModel) + Gewichte
model.save(RUN_DIR / "model.keras")
np.save(RUN_DIR / "y_test.npy", yte)
np.save(RUN_DIR / "y_proba.npy", y_proba)
with open(RUN_DIR / "config.json", "w") as f:
    json.dump({
        "ticker": TICKER, "start": START, "end": END, "interval": INTERVAL,
        "horizon": HORIZON, "lookback": LOOKBACK, "features": FEATURES,
        "scaler": "StandardScaler", "seed": SEED, "batch": BATCH, "epochs": EPOCHS
    }, f, indent=2)
print(f"\nArtefakte gespeichert in: {RUN_DIR}")


Artefakte gespeichert in: ..\results\2025-10-03_00-36-36_lstm
